In [115]:
!nvidia-smi

Wed Jun 23 06:41:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    38W / 300W |   1345MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [116]:
'''ResNet in PyTorch.

For Pre-activation ResNet, see 'preact_resnet.py'.

Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        # self.in_planes = 64
        self.in_planes = 64 
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

def ResNet50(num_classes=10):
    return ResNet(Bottleneck, [3,4,6,3], num_classes=num_classes)

def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])

def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])


def test():
    net = ResNet18()
    y = net(torch.randn(1,3,32,32))
    print(y.size())

# test()


In [117]:
import collections
orderedDict = collections.OrderedDict()


from collections import OrderedDict
model = nn.Sequential(OrderedDict([
        ('conv1',nn.Conv2d(1, 16, 4, stride=2, padding=1)),
        ('relu1',nn.ReLU()),
        ('conv2',nn.Conv2d(16, 32, 4, stride=2, padding=1)),
        ('relu2',nn.ReLU()),
        ('flatten',nn.Flatten()),
        ('linear1',nn.Linear(32*7*7,100)),
        ('relu3',nn.ReLU()),
        ('linear2',nn.Linear(100, 10))
     ]))

print(model)

Sequential(
  (conv1): Conv2d(1, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(16, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (relu2): ReLU()
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=1568, out_features=100, bias=True)
  (relu3): ReLU()
  (linear2): Linear(in_features=100, out_features=10, bias=True)
)


In [118]:
# def zero_gradients(x):
#      if isinstance(x, torch.Tensor):
#          if x.grad is not None:
#              x.grad.detach_()
#              x.grad.data.zero_()
#      elif isinstance(x, container_abcs.Iterable):
#          for elem in x:
#              zero_gradients(elem)
# # 
# !python /content/gdrive/MyDrive/improved_wasserstein/attack_mnist_baseline.py



In [119]:
# !pip install torchattacks
# !pip install geomloss
!rm -rf /content/gdrive/MyDrive/Colab_Notebooks/Images/MNIST
!rm -rf /content/gdrive/MyDrive/Colab_Notebooks/Images/MNIST.zip
!mkdir /content/gdrive/MyDrive/Colab_Notebooks/Images/MNIST

!python /content/gdrive/MyDrive/improved_wasserstein/attack_mnist_baseline.py
# !zip -r /content/gdrive/MyDrive/Colab_Notebooks/Images/MNIST.zip /content/gdrive/MyDrive/Colab_Notebooks/Images/MNIST

==> Preparing data..
==> Building model..
==> regularization set to 1000
norm is linfinity epsilon = 0.001 epsilon-factor =  0.001 alpha= 0.1
batch_idx 0 acc= 54.5
Runtime of the program is 52.3708598613739


In [120]:
#https://colab.research.google.com/github/kjamithash/Pytorch_DeepLearning_Experiments/
#blob/master/FashionMNIST_ResNet_TransferLearning.ipynb#scrollTo=Lsbcb_WgFx3c

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
from torchvision import transforms
import time
from tqdm.autonotebook import tqdm
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import inspect

import matplotlib.pyplot as plt
import numpy as np

# class MnistResNet(nn.Module):
#   def __init__(self, in_channels=1):
#     super(MnistResNet, self).__init__()

#     # Load a pretrained resnet model from torchvision.models in Pytorch
#     self.model = resnet18()

#     # Change the input layer to take Grayscale image, instead of RGB images. 
#     # Hence in_channels is set as 1 or 3 respectively
#     # original definition of the first layer on the ResNet class
#     # self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
#     self.model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
    
#     # Change the output layer to output 10 classes instead of 1000 classes
#     num_ftrs = self.model.fc.in_features
#     self.model.fc = nn.Linear(num_ftrs, 10)

#   def forward(self, x):
#     return self.model(x)


model = nn.Sequential(OrderedDict([
        ('conv1',nn.Conv2d(1, 16, 4, stride=2, padding=1)),
        ('relu1',nn.ReLU()),
        ('conv2',nn.Conv2d(16, 32, 4, stride=2, padding=1)),
        ('relu2',nn.ReLU()),
        ('flatten',nn.Flatten()),
        ('linear1',nn.Linear(32*7*7,100)),
        ('relu3',nn.ReLU()),
        ('linear2',nn.Linear(100, 10))
     ]))
net = model
# model_state_dict = torch.load("/content/gdrive/MyDrive/Colab_Notebooks/MnistCNN")
# model.load_state_dict(model_state_dict)


# input = torch.randn((16,1,244,244))
# output = my_resnet(input)
# print(output.shape)

# print(my_resnet)

if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")



# def get_data_loaders(train_batch_size, val_batch_size):
fashion_mnist = torchvision.datasets.FashionMNIST(download=True, train=True, root=".").train_data.float()

data_transform = transforms.Compose([ transforms.ToTensor(), 
                                      transforms.RandomAffine(30,translate=(0.0,0.0)), 
                                      transforms.Normalize((fashion_mnist.mean()/255,), (fashion_mnist.std()/255,))])

trainloader = DataLoader(torchvision.datasets.FashionMNIST(download=True, root=".", transform=data_transform, train=True),
                          batch_size=1, shuffle=True)
testset = torchvision.datasets.FashionMNIST(root=".", train=False, download= True , transform=data_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size= 200, shuffle= False )

val_loader = DataLoader(torchvision.datasets.FashionMNIST(download=False, root=".", transform=data_transform, train=False),
                        batch_size=1, shuffle=False)

    
    # return train_loader, val_loader, testloader


mu = (fashion_mnist.mean()/255).to(device)
std = (fashion_mnist.std()/255).to(device)
unnormalize = lambda x: x*std + mu
normalize = lambda x: (x-mu)/std
def calculate_metric(metric_fn, true_y, pred_y):
    if "average" in inspect.getfullargspec(metric_fn).args:
        return metric_fn(true_y, pred_y, average="macro")
    else:
        return metric_fn(true_y, pred_y)
    
def print_scores(p, r, f1, a, batch_size):
    for name, scores in zip(("precision", "recall", "F1", "accuracy"), (p, r, f1, a)):
        print(f"\t{name.rjust(14, ' ')}: {sum(scores)/batch_size:.4f}")

# model:
model = model.to(device)

# params you need to specify:
epochs = 10

# Dataloaders
# train_loader, val_loader, testloader = get_data_loaders(batch_size, batch_size)

# loss function and optimiyer
loss_function = nn.CrossEntropyLoss() # your loss function, cross entropy works well for multi-class problems

# optimizer, I've used Adadelta, as it wokrs well without any magic numbers
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4) # Using Karpathy's learning rate constant

start_ts = time.time()

losses = []
batches = len(trainloader)
val_batches = len(val_loader)

# loop for every epoch (training + evaluation)
# for epoch in range(epochs):
#     total_loss = 0

#     # progress bar (works in Jupyter notebook too!)
#     progress = tqdm(enumerate(train_loader), desc="Loss: ", total=batches)

#     # ----------------- TRAINING  -------------------- 
#     # set model to training
#     model.train()
#     total = 0
#     for i, data in progress:
#         X, y = data[0].to(device), data[1].to(device)
#         targets_attack = y
#         batch_idx = i
#         targets = y
#         total += targets.size(0)
#         # training step for single batch
#         inputs_pgd,y,  _ = attack(torch.clamp(unnormalize(X),min=0), 
#                                  targets_attack, model,  C, total,batch_idx,targets,
#                                  p= p, alpha=alpha , norm=norm ,
#                                  maxiters= 10 ,thold =10 , normalize=normalize ,
#                                  gamma = .5 ,
#                                  kernel_size= 784 , delta = 0.003 , 
#                                  theeta = 100 , tau =.01 , eta = .001)
#                                 #  eta = .25 )
#         model.zero_grad()
#         outputs = model(normalize(inputs_pgd))
#         loss = loss_function(outputs, y)
#         loss.backward()
#         optimizer.step()

#         # getting training quality data
#         current_loss = loss.item()
#         total_loss += current_loss

#         # updating progress bar
#         progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))
        
#     # releasing unceseccary memory in GPU
#     if torch.cuda.is_available():
#         torch.cuda.empty_cache()
    
#     # ----------------- VALIDATION  ----------------- 
#     val_losses = 0
#     precision, recall, f1, accuracy = [], [], [], []
    
#     # set model to evaluating (testing)
#     model.eval()
#     with torch.no_grad():
#         for i, data in enumerate(val_loader):
#             X, y = data[0].to(device), data[1].to(device)

#             outputs = model(X) # this get's the prediction from the network

#             val_losses += loss_function(outputs, y)

#             predicted_classes = torch.max(outputs, 1)[1] # get class from network's prediction
            
#             # calculate P/R/F1/A metrics for batch
#             for acc, metric in zip((precision, recall, f1, accuracy), 
#                                    (precision_score, recall_score, f1_score, accuracy_score)):
#                 acc.append(
#                     calculate_metric(metric, y.cpu(), predicted_classes.cpu())
#                 )
          
#     print(f"Epoch {epoch+1}/{epochs}, training loss: {total_loss/batches}, validation loss: {val_losses/val_batches}")
#     print_scores(precision, recall, f1, accuracy, val_batches)
#     losses.append(total_loss/batches) # for plotting learning curve
# print(f"Training time: {time.time()-start_ts}s")

            
# torch.save(model.state_dict(), "/content/gdrive/MyDrive/Colab_Notebooks/MnistCNN")    
# model = MnistCNN()
# model_state_dict = torch.load("/content/gdrive/MyDrive/Colab_Notebooks/MnistCNN")
# model.load_state_dict(model_state_dict)




/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:62: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [121]:
# # '''
# # Properly implemented ResNet-s for CIFAR10 as described in paper [1].
# # The implementation and structure of this file is hugely influenced by [2]
# # which is implemented for ImageNet and doesn't have option A for identity.
# # Moreover, most of the implementations on the web is copy-paste from
# # torchvision's resnet and has wrong number of params.
# # Proper ResNet-s for CIFAR10 (for fair comparision and etc.) has following
# # number of layers and parameters:
# # name      | layers | params
# # ResNet20  |    20  | 0.27M
# # ResNet32  |    32  | 0.46M
# # ResNet44  |    44  | 0.66M
# # ResNet56  |    56  | 0.85M
# # ResNet110 |   110  |  1.7M
# # ResNet1202|  1202  | 19.4m
# # which this implementation indeed has.
# # Reference:
# # [1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
# #     Deep Residual Learning for Image Recognition. arXiv:1512.03385
# # [2] https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py
# # If you use this implementation in you work, please don't forget to mention the
# # author, Yerlan Idelbayev.
# # '''
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.nn.init as init

# from torch.autograd import Variable

# __all__ = ['ResNet', 'resnet20', 'resnet32', 'resnet44', 'resnet56_custom', 'resnet110', 'resnet1202']

# def _weights_init(m):
#     classname = m.__class__.__name__
#     #print(classname)
#     if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
#         init.kaiming_normal_(m.weight)

# class LambdaLayer(nn.Module):
#     def __init__(self, lambd):
#         super(LambdaLayer, self).__init__()
#         self.lambd = lambd

#     def forward(self, x):
#         return self.lambd(x)


# class BasicBlock(nn.Module):
#     expansion = 1

#     def __init__(self, in_planes, planes, stride=1, option='A'):
#         super(BasicBlock, self).__init__()
#         self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
#         self.bn1 = nn.BatchNorm2d(planes)
#         self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
#         self.bn2 = nn.BatchNorm2d(planes)

#         self.shortcut = nn.Sequential()
#         if stride != 1 or in_planes != planes:
#             if option == 'A':
#                 """
#                 For CIFAR10 ResNet paper uses option A.
#                 """
#                 self.shortcut = LambdaLayer(lambda x:
#                                             F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
#             elif option == 'B':
#                 self.shortcut = nn.Sequential(
#                      nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
#                      nn.BatchNorm2d(self.expansion * planes)
#                 )

#     def forward(self, x):
#         out = F.relu(self.bn1(self.conv1(x)))
#         out = self.bn2(self.conv2(out))
#         out += self.shortcut(x)
#         out = F.relu(out)
#         return out


# class ResNet(nn.Module):
#     def __init__(self, block, num_blocks, num_classes=10):
#         super(ResNet, self).__init__()
#         self.in_planes = 16

#         self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
#         self.bn1 = nn.BatchNorm2d(16)
#         self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
#         self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
#         self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
#         self.linear = nn.Linear(64, num_classes)

#         self.apply(_weights_init)

#     def _make_layer(self, block, planes, num_blocks, stride):
#         strides = [stride] + [1]*(num_blocks-1)
#         layers = []
#         for stride in strides:
#             layers.append(block(self.in_planes, planes, stride))
#             self.in_planes = planes * block.expansion

#         return nn.Sequential(*layers)

#     def forward(self, x):
#         out = F.relu(self.bn1(self.conv1(x)))
#         out = self.layer1(out)
#         out = self.layer2(out)
#         out = self.layer3(out)
#         out = F.avg_pool2d(out, out.size()[3])
#         out = out.view(out.size(0), -1)
#         out = self.linear(out)
#         return out


# def resnet20():
#     return ResNet(BasicBlock, [3, 3, 3])


# def resnet32():
#     return ResNet(BasicBlock, [5, 5, 5])


# def resnet44():
#     return ResNet(BasicBlock, [7, 7, 7])


# def resnet56_custom():
#     return ResNet(BasicBlock, [9, 9, 9])


# def resnet110():
#     return ResNet(BasicBlock, [18, 18, 18])


# def resnet1202():
#     return ResNet(BasicBlock, [200, 200, 200])


# def test(net):
#     import numpy as np
#     total_params = 0

#     for x in filter(lambda p: p.requires_grad, net.parameters()):
#         total_params += np.prod(x.data.numpy().shape)
#     print("Total number of params", total_params)
#     print("Total layers", len(list(filter(lambda p: p.requires_grad and len(p.data.size())>1, net.parameters()))))


# if __name__ == "__main__":
#     for net_name in __all__:
#         if net_name.startswith('resnet'):
#             print(net_name)
#             test(globals()[net_name]())
#             print()

In [122]:

import numpy as np

def mat_min(*args):
  maatmin = np.min(mat,axis=(1,2))
  return maatmin

mat = np.random.randn(2,2,2)
print('ma',mat)
mmin = mat_min(mat)
print(mmin)

mat = np.array([[[1,2],[3,4]],[[5,6],[7,8]]])
print('new',mat)
mmin = mat_min(mat)
print(mmin)





ma [[[-0.27755712 -0.98029187]
  [ 0.26539887 -1.34501095]]

 [[ 1.21617634 -2.01023592]
  [-0.17088618 -0.03967752]]]
[-1.34501095 -2.01023592]
new [[[1 2]
  [3 4]]

 [[5 6]
  [7 8]]]
[1 5]


In [123]:
# import numpy as np
# import time
# import mouse

# while True:
#   random_row = np.random.random_sample()*100
#   random_col = np.random.random_sample()*10
#   random_time = np.random.random_sample()*np.random.random_sample() * 100
#   mouse.move(random_row, random_col, absolute=False, duration=0.2)
#   mouse.move(-random_row, -random_col, absolute=False, duration = 0.2)
#   mouse.LEFT
#   time.sleep(random_time)



In [124]:
# !unzip '/content/gdrive/MyDrive/Colab_Notebooks/PyTorch_CIFAR10-master.zip' -d '/content/gdrive/MyDrive/Colab_Notebooks/PyTorch_CIFAR10-master'


In [125]:
#!ls /content/gdrive/MyDrive/Colab_Notebooks/Images

#!ls /content/gdrive/MyDrive/projected_sinkhorn/checkpoints/

In [126]:
!rm -rf /content/gdrive/MyDrive/Colab_Notebooks/Images/MNIST
!rm -rf /content/gdrive/MyDrive/Colab_Notebooks/Images/MNIST.zip
!mkdir /content/gdrive/MyDrive/Colab_Notebooks/Images/MNIST

!rm -rf /content/gdrive/MyDrive/Colab_Notebooks/Images/fashionMNIST
!rm -rf /content/gdrive/MyDrive/Colab_Notebooks/Images/fashionMNIST.zip
!mkdir /content/gdrive/MyDrive/Colab_Notebooks/Images/fashionMNIST

!rm -rf /content/gdrive/MyDrive/Colab_Notebooks/Images/CIFAR
!rm -rf /content/gdrive/MyDrive/Colab_Notebooks/Images/CIFAR.zip
!mkdir /content/gdrive/MyDrive/Colab_Notebooks/Images/CIFAR
#!rm /content/gdrive/MyDrive/projected_sinkhorn/checkpoints/mnist_dual_epoch_*.pth

In [127]:
# !ls /content/gdrive/MyDrive/Colab_Notebooks/projected_sinkhorn-wong/checkpoints
#!ls /content/gdrive/MyDrive/improved_wasserstein/checkpoints
# !ls /content/gdrive/MyDrive/projected_sinkhorn/checkpoints

In [128]:
# import os
# os.chdir('/content/gdrive/wong_sinkhorn/projected_sinkhorn-master/')
# !pwd

In [129]:
# !python setup.py install

In [130]:
#!cp -r '/content/gdrive/MyDrive/improved_wasserstein/checkpoints/.' '/content/gdrive/MyDrive/Colab_Notebooks/projected_sinkhorn-wong/checkpoints'
#!python attack_mnist.py

In [131]:
import time

# starting time
start = time.time()

# program body starts
for i in range(200):
    print(i)


end = time.time()
print(f"Runtime of the program is {end - start}")


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
Runtime of the program is 0.018764972686767578


In [132]:
#!python /content/gdrive/MyDrive/improved_wasserstein/attack_cifar_baseline.py
!python /content/gdrive/MyDrive/improved_wasserstein/attack_mnist_baseline.py
# !zip -r /content/gdrive/MyDrive/Colab_Notebooks/Images/MNIST.zip /content/gdrive/MyDrive/Colab_Notebooks/Images/MNIST

==> Preparing data..
==> Building model..
==> regularization set to 1000
norm is linfinity epsilon = 0.001 epsilon-factor =  0.001 alpha= 0.1
batch_idx 0 acc= 54.5
Runtime of the program is 52.59829831123352


In [133]:
# !python /content/gdrive/MyDrive/Colab_Notebooks/projected_sinkhorn-wong/attack_mnist.py
# !zip -r /content/gdrive/MyDrive/Colab_Notebooks/Images/MNIST.zip /content/gdrive/MyDrive/Colab_Notebooks/Images/MNIST

In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math
import sys
import cv2
from PIL import Image
from torch.optim.lr_scheduler import StepLR
import time
from google.colab import drive
#drive.flush_and_unmount()

from google.colab import drive
#drive.mount('/content/gdrive/', force_remount=True)

# drive.mount('/content/gdrive/')
#!mkdir '/content/gdrive/MyDrive/Colab_Notebooks/drive/MyDrive/projected_sinkhorn/checkpoints'
#from google.colab import files
#uploaded = files.upload()

sys.path.append('/content/gdrive/MyDrive/Colab_Notebooks/drive/MyDrive/projected_sinkhorn/checkpoints/')
sys.path.append('/content/gdrive/MyDrive/Colab_Notebooks/drive/MyDrive/projected_sinkhorn/dataset/')
sys.path.append('/content/gdrive/MyDrive/Colab_Notebooks/drive/MyDrive/projected_sinkhorn/')
 
#from __future__ import print_function
import timeit 
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import os
import argparse
import gc
import imutils
import cv2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()
path = '/content/gdrive/MyDrive/Colab_Notebooks/Images/fashionMNIST'



Mounted at /content/gdrive/


In [135]:
torch.cuda.empty_cache()

# New Section

In [136]:
a = torch.randn(5,3,28,28)
za = a.view(-1,784)
# print(a)
# b=a.view(-1,2*2).min(axis=1)
# print(b)
Zmin, Zind = a.view(-1,784).min(axis=1)
Zmin = Zmin.unsqueeze(-1).unsqueeze(-1)
# Zmin = Zmin.transpose(1,0)
print(Zmin.size())
# K = Zmin.repeat(1,1,1,1)
# K=K.transpose(3,0)
# print(K.size())
za = za.add(-Zmin)

torch.Size([15, 1, 1])


In [137]:
c = np.random.rand(2,3,2,2)
print(c.shape[1])

3


In [138]:
# c = np.random.rand(2,1,2,2)
# d = np.array([1,2])
# print(c)
# f = c/d.reshape(2,1,1,1)
# print(f)
# c = torch.rand(2,2)
# print(c)
# d = c.unsqueeze(0)
# al = d.repeat(2,3,1,1)
# print(al)

# a = torch.randn(2,5)
# print(a)
# b,c=torch.max(a,dim=1)
# g= torch.max(b)
# print(g)


# c = torch.randn(tensor.size()).bool()
# print(c.size())
# c[0,3] = False
# val = c==True
# print(torch.sum(val))



# a = torch.randn(2,2,2,2)
# b = torch.randn(2,2,2,2)
# c = torch.subtract(a,b)
# print(a,b,c)


# g = torch.abs(a.reshape(4,-1) - b.reshape(4,-1))
# d=g[:,1]
# print('size',d.size())
# print('max is',torch.max(d))
# ind = d <.7
# print(ind)
# t = a
# print('t',t)
# t[ind,:,:] = 1
# print('t after = 1',t)
# a[ind,:,:] = t[ind,:,:]
# print('a = t',a)

    
#print(torch.sum(torch.mul(c,d),(1,2,3)))
# al = torch.diag_embed(c)
# print(al)

#c = np.random.rand(2,3,2,2)
#print(c)
# pixmax = np.max(c, axis = (2,3))
# print('pixmax',pixmax)
# pixmax = pixmax.reshape(2,3,1,1)
# print('pixmax  resh',pixmax)
# c = c/pixmax
# print('c o',c)

# d = c.view(2,3,-1)

# f = d.sum(-1)

# g = f.view(2,3,1,1)
# print(g)
# normalization = c.view(2,3,-1).sum(-1).view(2,3,1,1)
# print(c/normalization)

Basic Helper Functions

In [139]:

# def __init__(self, name=None):
#     self.name = name

def __enter__(self):
    self.tstart = time.time()

def __exit__(self, type, value, traceback):
    if self.name:
        print('[%s]' % self.name,)
    print('Elapsed: %s' % (time.time() - self.tstart))

In [140]:
def wasserstein_cost( p, kernel_size,n ):
    print('p is ',p)
    C = torch.zeros(kernel_size**2,kernel_size**2)
    xs, ys = torch.meshgrid([torch.arange(kernel_size), torch.arange(kernel_size)])
    a = xs.reshape(xs.size(0)**2,-1)
    b = ys.reshape(ys.size(0)**2,-1)
    c = torch.randn(xs.size(0)**2,2)*0
    c[:,0] = a.reshape(-1)
    c[:,1] = b.reshape(-1)
    for i in range(int(kernel_size)**2):
      for j in range(int(kernel_size)**2):
        C[i,j] = ( abs(c[i,0] - c[j,0])**p + abs(c[i,1] - c[j,1])**p)**(1/p)
    return C    
#     C = torch.zeros(kernel_size**2,kernel_size**2)
#     for i in range(kernel_size**2):
#       for j in range(kernel_size**2):
#         tix = i/n
#         tiy = i%n
#         tjx = j/n
#         tjy = j%n
#         C[i,j] = ( abs(tix - tjx)**p + abs(tiy - tjy)**p)**(1/p)
#     print(C)    
#     return C
    

def _expand(X, shape): 
    return X.view(*X.size()[:-1], *shape)

def unflatten2(X):
    n = X.size(-1)
    k = int(math.sqrt(n))
    return _expand(X,(k,k))

def unsqueeze3(X):
    return X.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)

def _expand_filter(X, nfilters , index ): 
    sizes = list(-1 for _ in range(X.dim()))
    sizes[-index] = nfilters
    return X.expand(*sizes)


def _unfold(x, kernel_size, padding=None): 
    size = x.size()
    if len(size) > 4: 
        x = x.contiguous().view(-1, *size[-3:])
    out = F.unfold(x, kernel_size, padding=kernel_size//2)
    if len(size) > 4: 
        out = out.view(*size[:-3], *out.size()[1:])
    return out

def collapse2(X): 
    return X.view(*X.size()[:-2], -1)

def _mm(A,x, shape): 
    kernel_size = A.size(-1) # 5
    nfilters = shape[1]  # 1
    unfolded = _unfold(x, kernel_size, padding=kernel_size//2).transpose(-1,-2)     
    unfolded = _expand(unfolded, (A.size(-3),A.size(-2)*A.size(-1))).transpose(-2,-3) #( 4 , 1 , 28*28  , 25 )
    out = torch.matmul(unfolded, collapse2(A.contiguous()).unsqueeze(-1)).squeeze(-1) #(4 , 1 , 25 , 1)
    return unflatten2(out)  #(4 , 1, 28*28 , 1) -> #(4 * 1 , 28 , 28)
#(728*728) * (728 , 4)
def any_nan(X): 
    return (X != X).any().item()

def print_cifar( pixels ):
    if pixels.shape[0] == 1:
      plt.imshow( pixels[0] , cmap = 'gray' )
      plt.show()
      return
    #print('Pixel shape' , pixels.shape)
    pixels = (pixels.transpose(1,2,0)*255).astype(np.uint8)
    # fig = plt.figure(figsize=(3,3)) 
    # ax = fig.add_subplot(131) 
    # ax.imshow(pixels,interpolation='nearest')
    # plt.show()
    # plt.axis('off')
    img = plt.imshow(pixels, interpolation='nearest')
    plt.axis('off')
  


def print_temp( pixels  ):
    torch.clamp(pixels , min = 0 , max = 1)
    pixels = pixels[0].cpu().detach().numpy()
    print_cifar(pixels)



Calculate ∇H∗qk(uk)

In [141]:
#function to calculate ∇H∗qk(uk) K - (4 , 1 , 5 , 5) alpha - (4 , 1 ,28 , 28)
def del_H( q , alpha , K  ):
  q1 = q/( _mm(K , alpha , alpha.size()) + 1e-6 )
  q1 = alpha* _mm(K , q1 , q1.size())
  return q1  
def alph( u , gamma):
  return torch.exp(u/gamma )

In [142]:
def new_del_H( q , alpha , K  ):
  return alpha*torch.matmul( q/(torch.matmul(alpha , K.t())) , K.t()) 
def alph( u , gamma):
  return torch.exp(u/gamma )

In [143]:
# class ResNet(torch.nn.Module):
#     def __init__(self, module):
#         super().__init__()
#         self.module = module

#     def forward(self, inputs):
#         return self.module(inputs) + inputs

In [144]:
def dual_attack( marginals , image ,t,batch_idx,thold,targets_attack,targets, W , C , gamma , delta , theeta , tau,eta):
  shape = marginals.size()
  N = shape[0]
  flag = True  
  flag1 = True 
  marginals = marginals.reshape(N,shape[1],-1)
  #print('max marginal is',torch.max(marginals))
  n = marginals.size(-1)
  #print('n is',n)
  W = W.reshape(N,shape[1],-1)        #flattening W
  #print('shape of W is',W.size())
  imgshape = image.size()     
  image = image.reshape(-1)  #flattening image
  with torch.no_grad():
    Vecone = W.new_ones((n))        # X -> ( u(k's) , v)  initialized with ones
    Vecone = Vecone.unsqueeze(-1)
    alpha = 0*W.new_ones((N,shape[1],n))
    #print('shape of alpha is',alpha.size())
    beta = 0*W.new_ones((N,shape[1],n))
    z = W
    #print('max z is',torch.max(z))
    K =  torch.exp(-C/eta).to(device)
    K = K.unsqueeze(0)
    K = K.repeat(N,shape[1],1,1)
    #print('size of K is ------',K.size())
    
    eps = 0.01
    co = 0
   
    delta = 0.05
    
    #while torch.sum(torch.mul(B/torch.sum(B),K),(1,2,3))>delta: #uncomment for attack
    while True: #comment after adversarial training
      
      z[z < 0] = 0 
      al = torch.diag_embed(torch.exp(alpha)/eta)
      be = torch.diag_embed(torch.exp(beta)/eta)
      B = torch.matmul(al,K)
      B = torch.matmul(B,be)
      
      #B = B/torch.sum(B)  
      #print('B size',B.size())
      r = torch.matmul(B,Vecone)
      Btrans = torch.transpose(B,2,3)
      #print('Btrans size',Btrans.size())
      q = torch.matmul(Btrans,Vecone)
      r[r < 0] = 0 
      q[q < 0] = 0  
      r = r.squeeze(-1) 
      q = q.squeeze(-1)  
      if co%2 == 0:
          alpha = (alpha/tau + torch.log(marginals + eps) - torch.log(r+eps))*eta*tau/(eta+tau)
          if any_nan(alpha):
            print('alpha got nan at itration ' , co )
      else:      
          beta = (beta/tau + torch.log(z+eps) - torch.log(q+eps))*eta*tau/(eta+tau)
          if any_nan(beta):
            print('beta got nan at itration ' , co )
            break
          # tz = z      
          # tz = W + (tau/gamma)*torch.exp(-beta/tau)  
          # dif = (torch.abs(tz.reshape(N,-1) - marginals.reshape(N,-1))).squeeze(-1)  
          # #print('OT is', torch.sum(torch.mul(B/torch.sum(B),K),(1,2,3)))  
          z[:,:,:] = W[:,:,:] + (tau/gamma)*torch.exp(-beta[:,:,:]/tau)
          # if shape[1]==1:
          Wbak = z.reshape(shape[0],shape[1],shape[2],shape[3])
          Wmin, Wind = Wbak.view(-1,shape[2]*shape[3]).min(axis=1)
          Wmin = Wmin.unsqueeze(-1).unsqueeze(-1)
          # print('size of z',z.size(),'size of Wmin',Wmin.size())
          z = z.add(-Wmin.reshape(shape[0],shape[1],-1))  
          OT = torch.sum(torch.mul(B/torch.sum(B),K),(1,2,3))
          # print('OT is....=',OT)
          if any_nan(z):
            print('z got nan at itration ' , co )
            print(z)
            break
      
      
    
      if co>thold and co%2 == 0:
        print('here........')
        break 
      elif co>thold:
        al = torch.diag_embed(torch.exp(alpha)/eta)
        be = torch.diag_embed(torch.exp(beta)/eta)
        B = torch.matmul(al,K)
        B = torch.matmul(B,be)
        break
      co+=1
    
    OT = torch.sum(torch.mul(B/torch.sum(B),K),(1,2,3))
    
    Z = z.reshape(shape[0],shape[1],shape[2],shape[3])
    # if shape[1]==1:
    Zmin, Zind = Z.view(-1,shape[2]*shape[3]).min(axis=1)
    Zmin = Zmin.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)
    Z = Z.add(-Zmin.reshape(shape[0],shape[1],1,1))
    
    M = marginals.reshape(shape[0],shape[1],shape[2],shape[3])

    # replace red and blue channel with original
    # Z[:,0,:,:] = M[:,0,:,:]
    # Z[:,2,:,:] = M[:,2,:,:]
    origimage = M[0].cpu().detach().numpy()
    # print('max of origimage',np.max(origimage),'min of origimage',np.min(origimage))
    origimage = 255*(origimage/np.max(origimage))

    advimage = Z[0].cpu().detach().numpy()
    # print('max of advimage',np.max(advimage),'min of advimage',np.min(advimage))
    advimage = advimage - np.min(advimage)
    # print('max of advimage',np.max(advimage),'min of advimage',np.min(advimage))
    advimage = 255*(advimage/np.max(advimage))

    diffimage = np.abs(origimage - advimage)
    diffimage = 255*(diffimage/np.max(diffimage))
    
    
    pixels = (origimage[0]).astype(np.uint8)
    # img = plt.imshow(pixels, interpolation='nearest')
    # plt.axis('off')

    pixels = (advimage[0]).astype(np.uint8)
    # img = plt.imshow(pixels, interpolation='nearest')
    # plt.axis('off')
  
    OTpixels = B[0].cpu().numpy()
   
    # OTpixels = np.log(OTpixels+1e-20)
    # print('sum=',np.sum(OTpixels),'max=',np.max(OTpixels),'min=',np.min(OTpixels))
    # OTpixels = OTpixels - np.min(OTpixels)
    # plt.imshow(OTpixels[0], cmap="gray",vmin=0, vmax=60) 
    # plt.show() 
    # print('sum=',np.sum(OTpixels),'max=',np.max(OTpixels),'min=',np.min(OTpixels))
    # OTpixels = OTpixels/np.max(OTpixels)
    
    # OTpixels = 255*OTpixels
    # print('sum=',np.sum(OTpixels),'max=',np.max(OTpixels),'min=',np.min(OTpixels))

    
    if origimage.shape[1] == 32:
      cv2.imwrite(os.path.join(path, 'orig'+str(batch_idx)+'.jpg'),(origimage.transpose(1,2,0)).astype(int))
      cv2.imwrite(os.path.join(path, 'attack_target'+'_'+str(batch_idx)+'targets_'+str(targets[0])+'targets_attack'+str(targets_attack[0])+'.jpg'),(advimage.transpose(1,2,0)).astype(int))
    else:
      cv2.imwrite(os.path.join(path, 'OT'+str(batch_idx)+'.jpg'),(OTpixels[0]).astype(int))
      # cv2.imwrite(os.path.join(path, 'orig'+str(batch_idx)+'.jpg'),(origimage[0]).astype(int))
      # cv2.imwrite(os.path.join(path, 'attack_'+str(batch_idx)+'_'+'gamma_'+str(gamma)+'_'+'.jpg'),(advimage[0]).astype(int))
    
    # cv2.imwrite(os.path.join(path, 'error_'+str(batch_idx)+str(t)+'.jpg'),(diffimage.transpose(1,2,0)).astype(int))
    # cv2.imwrite(os.path.join(path, 'OT_'+str(batch_idx)+str(t)+'.jpg'),(255*OTpixels[0]).astype(int))

       
  # del alpha,beta
  # gc.collect()  
  return Z, OT, B/torch.max(B)   #returning p

In [145]:
from matplotlib import pyplot as plt

def attack(XX ,yy ,  net ,C,total,batch_idx, targets,thold,alpha, p = 2 ,  kernel_size=5, maxiters= 10, 
         xmin=0, xmax=1, normalize=lambda x: x, verbose=0, 
             norm='linfinity' ,   gamma = 0.5 , delta = 0.01 , theeta = 1 , tau = 0.1 , eta = 100  ): 
    N = XX.size(0)
    X = XX#specify later
    y = yy
    #path = '/Users/madhushree/Desktop/drive/MyDrive/projected_sinkhorn/dataset/MNIST/'
    #normalization = XX.view(N,3,-1).sum(-1).view(N,3,1,1)  #Recheck if needed
    #normalization = XX.view(N,-1).sum(-1).view(N,1,1,1)  #Recheck if needed
    normalization = 255
    #print('normalization',normalization)
    X_ = X.clone()
    X_best = X.clone()
    err_best = err = net(normalize(X)).max(1)[1] != y
    t = 0
    eps = 0.01
    pix = X_.cpu().detach().numpy()
    yuniform = 0.1*torch.ones(N,10).to(device)
    yuniform[:,0]=0.1
    yuniform = yuniform/torch.sum(yuniform,0)
    # print('y',y,'yuniform',yuniform)
    #print('max of X is',np.max(pix))
    while True: 
        X_.requires_grad = True

        # # codde for sinkhorn loss
        # H_ = net(normalize(X_))
        # # H_.requires_grad = True
        # optSink = optim.SGD([X_], lr=0.1)
        # lossSinkhorn = SamplesLoss(loss="sinkhorn", p=1, blur=.5)
        # # print('H size',H_.size(),'yuniform size',yuniform.size())
        # Lsink = lossSinkhorn(H_,yuniform)
        # optSink.zero_grad()
        # Lsink.backward()

        
        opt = optim.SGD([X_], lr=0.1)
        loss = nn.CrossEntropyLoss()(net(normalize(X_)),y)
        opt.zero_grad()
        loss.backward()

        with torch.no_grad(): 
            # take a step
            
            if norm == 'linfinity': 
                X_[~err] += alpha*torch.sign(X_.grad[~err])
            elif norm == 'l2': 
                X_[~err] += (alpha*X_.grad/(X_.grad.view(X.size(0),-1).norm(dim=1).view(X.size(0),1,1,1)) + eps)[~err]
            else: 
                raise ValueError("Unknown norm")

            
            # project onto ball
            #X_[~err] = ( dual_attack( XX.clone()*normalization , 
#                                      X.clone()*normalization ,thold, 
#                                      X_.detach()*normalization , C , 
#                                      gamma , delta , theeta , tau,eta)/
#                                      normalization)[~err]
           
            # X_[~err], OT, D = dual_attack(XX.clone()*normalization , 
            #                          X.clone()*normalization,thold, 
            #                          X_.detach()*normalization , C , 
            #                          gamma , delta , theeta , tau,eta)
            Xall, OT, D = dual_attack(XX.clone()*normalization , 
                                     X.clone()*normalization, t,batch_idx,thold,y,targets,
                                     X_.detach()*normalization , C , 
                                     gamma , delta , theeta , tau,eta)
            X_[~err] = Xall[~err]
            X_[~err] = X_[~err]/normalization
            #X_[err] = X_[err]/normalization
            #What Normalizations to perform ?
            # print('Showing original image at iter',t)
            pixelsorig = X[0].cpu().detach().numpy()
            # #pixelsorig1 = X[1].cpu().detach().numpy()
            # # print_cifar(pixelsorig)
            # img = plt.imshow((pixelsorig.transpose(1,2,0)*255).astype(np.uint8), interpolation='nearest')
            # plt.axis('off')

            
            # print('X_ mins....',np.min(X_[0][0].cpu().detach().numpy()))
            # print('X_ max....',np.max(X_[0][0].cpu().detach().numpy()))
            # print('X_ sum....',np.mean(X_[0][0].cpu().detach().numpy()))
            pix = X_.cpu().detach().numpy()
            pixmax = np.max(pix, axis = (2,3))
            #print(pixmax)
            pixmin = np.min(pix, axis = (2,3))
            #--------
            pixm = pixmax[~err.cpu()]
            #print('pixm is......',pixm,'pixmax is..',pixmax)
            pixm = pixm.reshape(pixm.shape[0],pixm.shape[1],1,1)
            Xa= X_[~err].cpu()/pixm
            # X_[~err] = Xa.to(device)
            #print(err)
            #---------
            # print('Showing adver image at second',t)
            # pixelsadv = X_[0].cpu().detach().numpy()
            # pixelsadv1 = X_[1].cpu().detach().numpy()
            # print_cifar(pixelsadv)

            #print('pixmax is',pixmax)


            X_ = torch.clamp(X_, min=xmin, max=xmax)  #Recheck later if clamp is needed or not
            
            
       
            # print('Showing adv image at iter ',t)
            pixelsadv = X_[0].cpu().detach().numpy()
            # # print_cifar(pixelsadv)
            # img = plt.imshow((pixelsadv.transpose(1,2,0)*255).astype(np.uint8), interpolation='nearest')
            # print('max of pixeladv is',np.max(pixelsadv),'min of pixeladv is',np.min(pixelsadv))
            # print('max of pixelsorig is',np.max(pixelsorig),'min of pixelsorig is',np.min(pixelsorig))
            # #plt.axis('off')
           

            index = range(5)
            pixelsorig = X.cpu().numpy()
            pixelsadv = X_.cpu().numpy()
            error = (pixelsorig-pixelsadv)
            error = error - np.min(error)
            error = error/np.max(error)
            
            #print('Showing difference after Wasserstein attack')
            # difX0 = np.abs(X_[0].cpu().detach().numpy() - X[0].cpu().detach().numpy())
            # print('max of dif is',np.max(difX0),'summ of X_[0] is',np.sum(X_[0].cpu().detach().numpy()),
            #       'sum of X[0] is',np.sum(X[0].cpu().detach().numpy()))
            # difX0 = difX0/(np.max(difX0)+0.0001)
            # print('max of dif is',np.max(difX0))

            # difX1 = np.abs(X_[1].cpu().detach().numpy() - X[1].cpu().detach().numpy())
            # print('max of dif is',np.max(difX1),'max of X_[1] is',np.sum(X_[1].cpu().detach().numpy()),
            #       'max of X[1] is',np.sum(X[1].cpu().detach().numpy()))
            # difX1 = difX1/(np.max(difX1)+0.0001)
            #print_cifar(difX)
            err = (net(normalize(X_)).max(1)[1] != y)
            predicted_class = net(normalize(X_)).max(1)[1]
            flag = True
            # for i in index:
            #   # print('pred is..',predicted_class[i],'err',err[i])
            #   if pixelsorig.shape[1]==3:
            #     cv2.imwrite(os.path.join(path, 'orig_'+str(i)+'.jpg'),(255*pixelsorig[i].transpose(1,2,0)).astype(int))
            #     if (err[i] == True):
            #       cv2.imwrite(os.path.join(path, 'attack_'+str(i)+'.jpg'),(255*pixelsadv[i].transpose(1,2,0)).astype(int))
            #       cv2.imwrite(os.path.join(path, 'error_'+str(i)+'.jpg'),(255*error[i].transpose(1,2,0)).astype(int))
            #   if pixelsorig.shape[1]==1:
            #     if t < 3:
            #       cv2.imwrite(os.path.join(path, 'orig_'+str(i)+'_class_'+str(targets[i])+'.jpg'),(255*pixelsorig[i][0]).astype(int))
            #       if (err[i] == True):
            #         cv2.imwrite(os.path.join(path, 'attack_'+str(i)+'_class_'+str(targets[i])+'.jpg'),(255*pixelsadv[i][0]).astype(int))
            #         cv2.imwrite(os.path.join(path, 'error_'+str(i)+'_class_'+str(targets[i])+'.jpg'),(255*error[i][0]).astype(int))
            # #cv2.imwrite(os.path.join(path, 'OT_'+str(i)+'.jpg'),(255*OTpixels[0]).astype(int))
            err_rate = err.sum().item()
             
            #print('diff = ......', torch.max(torch.abs(X_ - X)) )
            if err_rate > err_best.sum().item():
                X_best = X_.clone() 
                err_best = err
            
            t += 1
            if err_rate == N or t == maxiters: 
                break

    torch.cuda.empty_cache()
    return X_best, y,err_best

Formulation

In [146]:
p = 1
kernel_size = 28
C = wasserstein_cost(p=p, kernel_size= kernel_size, n =  kernel_size)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
alpha = 0.02
norm = 'linfinity'
model_state_dict = torch.load("/content/gdrive/MyDrive/Colab_Notebooks/MnistCNN")
model.load_state_dict(model_state_dict,strict=False)
# model_state_dict = torch.load("/content/gdrive/MyDrive/projected_sinkhorn/checkpoints/fashionmnist_dual_epoch_50_advacc_37.62_accuracy_82.09.pth")
# model.load_state_dict(model_state_dict['model'])
start_epoch = 51
checkpoint_dir = '/content/gdrive/MyDrive/projected_sinkhorn/checkpoints'
checkpoint_file = '{}/fashionmnist_dual_epoch_{}_advacc_{}_accuracy_{}.pth'.format(checkpoint_dir,'{}','{}','{}')
for pr in model.parameters(): 
    pr.requires_grad = True
def trainfashion(epoch):
    print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    nominal_correct = 0
    total_epsilon = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        targets_attack = targets
        inputs_pgd,y, _ = attack(torch.clamp(unnormalize(inputs),min=0), 
                                 targets_attack, model,  C, total,batch_idx,targets,
                                 p= p, alpha=alpha , norm=norm ,
                                 maxiters= 20 ,thold = 10 , normalize=normalize ,
                                 gamma = .5 ,
                                 kernel_size= 1024 , delta = 0.003 , 
                                 theeta = 100 , tau = .01 , 
                                 eta = .001 )
        optimizer.zero_grad()
        outputs = model(normalize(inputs_pgd))
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        with torch.no_grad(): 
            outputs_nominal = model(inputs)
            _, predicted_nominal = outputs_nominal.max(1)
            nominal_correct += predicted_nominal.eq(targets).sum().item()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
           
            
        print(batch_idx, len(trainloader), 'Loss: %.3f | Adv Acc: %.3f%% (%d/%d) | Acc: %.3f%% (%d/%d) '
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total, 
                100.*nominal_correct/total, nominal_correct, total))
        # if batch_idx == 0:
        #   break
   
    advacc = np.round(100.*correct/total,2)
    acc = np.round(100.*nominal_correct/total,2)
    state = {
        'model': model.state_dict(),
        'acc': acc,
        'advacc': advacc,
        'epoch': epoch,
    }
    torch.save(state, checkpoint_file.format(epoch,advacc,acc))
    # import shutil 
    # shutil.copy(checkpoint_file.format(epoch,advacc,acc), '/content/gdrive/MyDrive/projected_sinkhorn/checkpoints/')



# model_state_dict = torch.load("/content/gdrive/MyDrive/Colab_Notebooks/MnistadvCNN")
# model.load_state_dict(model_state_dict)

def testfashionmnist():
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
           
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
    acc = 100.*correct/total
    print('Accuracy = ', acc )

def test_attackfashionmnist(): 
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    batchOT = 0
    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs.to(device), targets.to(device)
        targets_attack = targets # should add or subtract for targeted attack
        
        N = inputs.size(0)
        total += targets.size(0)
        inputs_pgd,y,  _ = attack(torch.clamp(unnormalize(inputs),min=0), 
                                 targets_attack, model,  C, total,batch_idx,targets,
                                 p= p, alpha=alpha , norm=norm ,
                                 maxiters= 10 ,thold =10 , normalize=normalize ,
                                 gamma = .5 ,
                                 kernel_size= 784 , delta = 0.003 , 
                                 theeta = 100 , tau =.01 , eta = .001)
                                #  eta = .25 )

        outputs_pgd = model(normalize(inputs_pgd))
        loss = criterion(outputs_pgd, y)

        test_loss += loss.item()
        _, predicted = outputs_pgd.max(1)
        
    
        # pixelsorig = inputs.cpu().numpy()
        # pixelsadv = inputs_pgd.cpu().numpy()
        # error = (pixelsorig-pixelsadv)
        # error = error - np.min(error)
        # error = error/np.max(error)
        # index = range(5)
        # for i in index:
            # cv2.imwrite(os.path.join(path, 'orig_'+str(i)+'.jpg'),(255*pixelsorig[i][0]).astype(int))
            # cv2.imwrite(os.path.join(path, 'attack_'+str(i)+'.jpg'),(255*pixelsadv[i][0]).astype(int))
            # cv2.imwrite(os.path.join(path, 'error_'+str(i)+'.jpg'),(255*error[i][0]).astype(int))

        correct += predicted.eq(targets).sum().item()
        print('Batch ' , batch_idx , 'Loss = ' , test_loss/(batch_idx+1) , 'Accuracy = ' , 100.*correct/total  )
        # print('original labels:predicted labels',targets,predicted)
        if batch_idx==20:
          break

        acc = 100.*correct/total
#         if batch_idx > 0:
#           break
    print('final accuracy = ' , 100.*correct/total)

#print('==> Evaluating model..')

# for epoch in range(start_epoch, start_epoch+30):
#     print('traaining start......')
#     test_attackfashionmnist()
#     testfashionmnist()
#     trainfashion(epoch)
#     testmnist()
# # print('==> Attacking model..')
# test_attackfashionmnist()

# model_state_dict  = torch.load("/content/gdrive/MyDrive/projected_sinkhorn/checkpoints/fashionmnist_dual_epoch_57_advacc_38.43_accuracy_82.19.pth")
# net.load_state_dict(model_state_dict['model'])
net = net.to(device)
# testfashionmnist()


# model_state_dict = torch.load("/content/gdrive/MyDrive/Colab_Notebooks/MnistCNN")
# net.load_state_dict(model_state_dict)
net = net.to(device)
# testfashionmnist()
!zip -r /content/gdrive/MyDrive/Colab_Notebooks/Images/fashionMNIST.zip /content/gdrive/MyDrive/Colab_Notebooks/Images/fashionMNIST    

p is  1
  adding: content/gdrive/MyDrive/Colab_Notebooks/Images/fashionMNIST/ (stored 0%)


 Gradient Sign attack which calls barycentre_attack

*   List item
*   List item



*   List item
*   List item



In [147]:
# ImageNet = torchvision.datasets.ImageNet(download=False, train=False, root=".").train_data.float()

# data_transform = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

# trainloader = DataLoader(torchvision.datasets.ImageNet(download=False, root=".", transform=data_transform, train=True),
#                           batch_size=1, shuffle=True)
# testset = torchvision.datasets.ImageNet(root=".", train=False, download= True , transform=data_transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size= 4, shuffle= False )


def testimagenet():
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
           
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
    acc = 100.*correct/total
    print('Accuracy = ', acc )

def test_attacktestimagenet(): 
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    batchOT = 0
    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs.to(device), targets.to(device)
        targets_attack = targets # should add or subtract for targeted attack
        
        N = inputs.size(0)
        total += targets.size(0)
        inputs_pgd,y,  _ = attack(torch.clamp(unnormalize(inputs),min=0), 
                                 targets_attack, model,  C, total,batch_idx,targets,
                                 p= p, alpha=alpha , norm=norm ,
                                 maxiters= 1 ,thold =10 , normalize=normalize ,
                                 gamma = .5 ,
                                 kernel_size= 224*224 , delta = 0.003 , 
                                 theeta = 100 , tau =.01 , eta = .001)
                                #  eta = .25 )

        outputs_pgd = model(normalize(inputs_pgd))
        loss = criterion(outputs_pgd, y)

        test_loss += loss.item()
        _, predicted = outputs_pgd.max(1)
        
    
        # pixelsorig = inputs.cpu().numpy()
        # pixelsadv = inputs_pgd.cpu().numpy()
        # error = (pixelsorig-pixelsadv)
        # error = error - np.min(error)
        # error = error/np.max(error)
        # index = range(5)
        # for i in index:
            # cv2.imwrite(os.path.join(path, 'orig_'+str(i)+'.jpg'),(255*pixelsorig[i][0]).astype(int))
            # cv2.imwrite(os.path.join(path, 'attack_'+str(i)+'.jpg'),(255*pixelsadv[i][0]).astype(int))
            # cv2.imwrite(os.path.join(path, 'error_'+str(i)+'.jpg'),(255*error[i][0]).astype(int))

        correct += predicted.eq(targets).sum().item()
        print('Batch ' , batch_idx , 'Loss = ' , test_loss/(batch_idx+1) , 'Accuracy = ' , 100.*correct/total  )
        # print('original labels:predicted labels',targets,predicted)
        if batch_idx==0:
          break

        acc = 100.*correct/total
#         if batch_idx > 0:
#           break
    print('final accuracy = ' , 100.*correct/total)

#print('==> Evaluating model..')

# for epoch in range(start_epoch, start_epoch+30):
#     print('traaining start......')
#     test_attackfashionmnist()
#     testfashionmnist()
#     trainfashion(epoch)
#     testmnist()
# # print('==> Attacking model..')
# test_attackfashionmnist()

# net = models.resnet18(pretrained=True)
# net = net.to(device)
# testfashionmnist()



# testfashionmnist()
!zip -r /content/gdrive/MyDrive/Colab_Notebooks/Images/fashionMNIST.zip /content/gdrive/MyDrive/Colab_Notebooks/Images/fashionMNIST    

updating: content/gdrive/MyDrive/Colab_Notebooks/Images/fashionMNIST/ (stored 0%)


 Gradient Sign attack which calls barycentre_attack

In [148]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.ToTensor(),])

transform_test = transforms.Compose([
    transforms.RandomAffine(0,translate=(0,0)),                            
    transforms.ToTensor(), ])

trainset = torchvision.datasets.MNIST(root='/content/gdrive/MyDrive/Colab_Notebooks/drive/MyDrive/projected_sinkhorn/dataset/data/', train=True, download=  True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True, num_workers=2)   # MNIST

dig = 5 #The digit to attack
testset = torchvision.datasets.MNIST(root='/content/gdrive/MyDrive/Colab_Notebooks/drive/MyDrive/projected_sinkhorn/dataset/data/', train=False, download= True , transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size= 200, shuffle= False )


print('==> Building model..')

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

#uncomment for mnist

net = nn.Sequential(
        nn.Conv2d(1, 16, 4, stride=2, padding=1),
        nn.ReLU(),
        nn.Conv2d(16, 32, 4, stride=2, padding=1),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(32*7*7,100),
        nn.ReLU(),
        nn.Linear(100, 10)
    )


netattack=net

==> Preparing data..
==> Building model..


Test on MNIST

In [149]:

#uncomment for mnist

# if device == 'cuda':
#     net = torch.nn.DataParallel(net)
#     cudnn.benchmark = True  
net = net.to(device)
netattack = netattack.to(device)
checkpoint_dir = '/content/gdrive/MyDrive/Colab_Notebooks/drive/MyDrive/projected_sinkhorn/checkpoints'
checkpoint_file = '{}/cifar_dual_epoch_{}_advacc_{}_acc_{}.pth'.format(checkpoint_dir,'{}','{}','{}')

alpha = 0.02
norm = 'linfinity'

# for mnist
#model_name = '/content/gdrive/MyDrive/Colab_Notebooks/drive/MyDrive/projected_sinkhorn/checkpoints/mnist_dual_epoch_39_advacc_92.86_acc_99.8_OT_3.643.pth'
#for cifar
#model_name = '/content/gdrive/MyDrive/Colab_Notebooks/drive/MyDrive/projected_sinkhorn/checkpoints/cifar_vanilla.pth'
# model_name = '/content/gdrive/MyDrive/Colab_Notebooks/projected_sinkhorn-wong/checkpoints/mnist_adv_training_wong.pth'
# model_name = '/content/gdrive/MyDrive/Colab_Notebooks/projected_sinkhorn-wong/checkpoints/mnist_adv_training_ours.pth'

#https://github.com/akamaster/pytorch_resnet_cifar10

# model_name = '/content/gdrive/MyDrive/Colab_Notebooks/resnet56-4bfd9763.th'
# model_name = '/content/gdrive/MyDrive/projected_sinkhorn/checkpoints/cifar_dual_epoch_5_advacc_58.57_acc_99.98_OT_0.004999999888241291.pth'
# model_name = '/content/gdrive/MyDrive/Colab_Notebooks/projected_sinkhorn-wong/checkpoints/cifar_adv_training_wong.pth'
model_name = '/content/gdrive/MyDrive/projected_sinkhorn/checkpoints/mnist_vanilla.pth'

# model_name = '/content/gdrive/MyDrive/projected_sinkhorn/checkpoints/mnist_dual_epoch_59_advacc_88.24_acc_98.31_OT_3.917.pth'
# model_name = '/content/gdrive/MyDrive/projected_sinkhorn/checkpoints/mnist_linf_robust.pth'
#model_name = '/content/gdrive/MyDrive/projected_sinkhorn/checkpoints/cifar_dual_epoch_0_advacc_58.96_acc_94.77_OT_0.02500000037252903.pth'
# model_name  = '/content/gdrive/MyDrive/projected_sinkhorn/checkpoints/mnist_dual_epoch_5_advacc_83.96_acc_99.69_OT_3.643.pth'
# model_name  = '/content/gdrive/MyDrive/projected_sinkhorn/checkpoints/mnist_dual_epoch_9_advacc_87.21_acc_99.71_OT_3.643.pth'
# model_name  = '/content/gdrive/MyDrive/projected_sinkhorn/checkpoints/mnist_dual_epoch_15_advacc_89.56_acc_99.74_OT_3.643.pth'


# model_name  = '/content/gdrive/MyDrive/projected_sinkhorn/checkpoints/cifar_dual_epoch_6_advacc_6.05_acc_99.97_OT_0.00800000037997961.pth'

# model_name  = '/content/gdrive/MyDrive/projected_sinkhorn/checkpoints/mnist_dual_epoch_39_advacc_85.66_acc_98.08_OT_3.917.pth'



binarize = False

# print('==> loading model {}'.format(model_name))

#uncomment it for mnist vanilla
# d = torch.load(model_name, map_location = 'cpu')

# # comment this for mnist
# d = torch.load(model_name)
# if 'state_dict' in d: 
#     net.load_state_dict(d['state_dict'][0])    
# elif 'robust' in model_name: 
#     net.load_state_dict(d)
# else: 
#     net.load_state_dict(d['net'])


# uncomment for adversarial model of wong
# checkpoint = torch.load(model_name)
# for key in list(checkpoint['net'].keys()):
#   new_key = key[7:]
#   checkpoint['net'][new_key] = checkpoint['net'].pop(key)
# net.load_state_dict(checkpoint['net'],strict=False)
# net = net.to(device)



criterion = nn.CrossEntropyLoss()

def testmnist():
    netattack.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            if binarize:
                inputs = (inputs >= 0.5).float()
            outputs = netattack(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    acc = 100.*correct/total
    print('Accuracy = ', acc )


# model_name = '/content/gdrive/MyDrive/Colab_Notebooks/projected_sinkhorn-wong/checkpoints/mnist_adv_training_ours.pth'
# checkpoint = torch.load(model_name)
# for key in list(checkpoint['net'].keys()):
#   new_key = key[7:]
#   checkpoint['net'][new_key] = checkpoint['net'].pop(key)
# net.load_state_dict(checkpoint['net'],strict=False)
# net = net.to(device)
# testmnist()

# model_name = '/content/gdrive/MyDrive/Colab_Notebooks/projected_sinkhorn-wong/checkpoints/mnist_adv_training_wong.pth'
# checkpoint = torch.load(model_name)
# for key in list(checkpoint['net'].keys()):
#   new_key = key[7:]
#   checkpoint['net'][new_key] = checkpoint['net'].pop(key)
# net.load_state_dict(checkpoint['net'],strict=False)
# net = net.to(device)
# testmnist()





In [150]:
normalize = lambda x: x #comment for cifar
p = 1
kernel_size = 28
C = wasserstein_cost(p=p, kernel_size= kernel_size, n =  kernel_size)

def test_attackmnist(): 
    netattack.eval()
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    batchOT = 0
    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs.to(device), targets.to(device)
        targets_attack = targets # should add or subtract for targeted attack
        if binarize: 
            inputs = (inputs >= 0.5).float()
        N = inputs.size(0)
        total += targets.size(0)
        inputs_pgd,y,  _ = attack(torch.clamp((inputs),min=0), 
                                 targets_attack, netattack,  C, total,batch_idx,targets,
                                 p= p, alpha=alpha , norm=norm ,
                                 maxiters= 10 ,thold =10 , normalize=normalize ,
                                 gamma = .5 ,
                                 kernel_size= 784 , delta = 0.003 , 
                                 theeta = 100 , tau =.01 , eta = .001)
                                #  eta = .25 )

        outputs_pgd = net(inputs_pgd)
        loss = criterion(outputs_pgd, y)

        test_loss += loss.item()
        _, predicted = outputs_pgd.max(1)
        
    
        # pixelsorig = inputs.cpu().numpy()
        # pixelsadv = inputs_pgd.cpu().numpy()
        # error = (pixelsorig-pixelsadv)
        # error = error - np.min(error)
        # error = error/np.max(error)
        # index = range(5)
        # for i in index:
            # cv2.imwrite(os.path.join(path, 'orig_'+str(i)+'.jpg'),(255*pixelsorig[i][0]).astype(int))
            # cv2.imwrite(os.path.join(path, 'attack_'+str(i)+'.jpg'),(255*pixelsadv[i][0]).astype(int))
            # cv2.imwrite(os.path.join(path, 'error_'+str(i)+'.jpg'),(255*error[i][0]).astype(int))

        correct += predicted.eq(targets).sum().item()
        print('Batch ' , batch_idx , 'Loss = ' , test_loss/(batch_idx+1) , 'Accuracy = ' , 100.*correct/total  )
        # print('original labels:predicted labels',targets,predicted)
        if batch_idx==0:
          break

        acc = 100.*correct/total
#         if batch_idx > 0:
#           break
    print('final accuracy = ' , 100.*correct/total)

#print('==> Evaluating model..')

# # print('==> Attacking model..')
start = time.time()
test_attackmnist()
end = time.time()
print(f"Runtime of the program is {end - start}")

# !zip -r /content/gdrive/MyDrive/Colab_Notebooks/Images/MNIST.zip /content/gdrive/MyDrive/Colab_Notebooks/Images/MNIST

p is  1
Batch  0 Loss =  2.307671546936035 Accuracy =  0.0
final accuracy =  0.0
Runtime of the program is 3.834124803543091


In [151]:
# !zip -r /content/gdrive/MyDrive/Colab_Notebooks/Images/MNIST.zip /content/gdrive/MyDrive/Colab_Notebooks/Images/MNIST

In [152]:
# Training
start_epoch = 0 # start from epoch 0 or last checkpoint epoch

optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
# optimizer = optim.SGD(net.parameters(), lr=.5, momentum=0.9)

# scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
for pr in net.parameters(): 
    pr.requires_grad = True
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    nominal_correct = 0
    total_epsilon = 0
    total = 0
    batchOT = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        targets_attack = targets
        # inputs_pgd,y, OT, _ = attack(torch.clamp((inputs),min=0), 
        #                          targets_attack, net,  C, total,batch_idx,targets,
        #                          p= p, alpha=alpha , norm=norm ,
        #                          maxiters= 20 ,thold = 10 , normalize=normalize ,
        #                          gamma = .5 ,
        #                          kernel_size= 1024 , delta = 0.003 , 
        #                          theeta = 100 , tau = .01 , 
        #                          eta = .001 )
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        with torch.no_grad(): 
            outputs_nominal = net(inputs)
            _, predicted_nominal = outputs_nominal.max(1)
            nominal_correct += predicted_nominal.eq(targets).sum().item()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
           
            
        print(batch_idx, len(trainloader), 'Loss: %.3f | Adv Acc: %.3f%% (%d/%d) | Acc: %.3f%% (%d/%d) '
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total, 
                100.*nominal_correct/total, nominal_correct, total))
        # if batch_idx == 2:
        #   break
   
    advacc = np.round(100.*correct/total,2)
    acc = np.round(100.*nominal_correct/total,2)
    state = {
        'net': net.state_dict(),
        'acc': acc,
        'advacc': advacc,
        'epoch': epoch,
    }
    torch.save(state, checkpoint_file.format(epoch,advacc,acc))
    import shutil 
    shutil.copy(checkpoint_file.format(epoch,advacc,acc), '/content/gdrive/MyDrive/projected_sinkhorn/checkpoints/')

def test(epoch):
    net.eval()
    test_loss = 0
    correct = 0
    nominal_correct = 0
    total = 0
    total_epsilon = 0
    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs.to(device), targets.to(device)
        inputs_pgd, _, epsilons = attack(torch.clamp((inputs),min=0), 
                                 targets, net,  C, total,
                                 p= p, alpha=alpha , norm=norm ,
                                 maxiters= 2 ,thold = 30 , normalize=normalize ,
                                 gamma = 2000 ,
                                 kernel_size= 1024 , delta = 0.003 , 
                                 theeta = 100 , tau = 5 ,
                                 eta = .5 )
        with torch.no_grad():
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            outputs_nominal = net(inputs)
            _, predicted_nominal = outputs_nominal.max(1)
            nominal_correct += predicted_nominal.eq(targets).sum().item()

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            total_epsilon += epsilons.sum().item()
            

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d) | Acc: %.3f%% (%d/%d) | Eps: %.3f%%'
                % (test_loss/(batch_idx+1), 100.*correct/total, correct, total, 
                100.*nominal_correct/total, nominal_correct, total, total_epsilon/total))

    # Save checkpoint.
    # acc = 100.*correct/total
    # avg_epsilon = total_epsilon/total
    
    state = {
        'net': net.state_dict(),
        'acc': acc,
        'eps': avg_epsilon,
        'epoch': epoch,
    }
    if not os.path.isdir(checkpoint_dir):
        os.mkdir(checkpoint_dir)
    torch.save(state, checkpoint_file.format(epoch))


# for epoch in range(start_epoch, start_epoch+):
#     print('traaining start......')
#     train(epoch)
#     testmnist()
#     #test(epoch)
    #acc = 100.*correct/total
    


In [153]:
# import os
# import argparse
# import sys
# #sys.path.append('drive/MyDrive/projected_sinkhorn/')
# # sys.path.append('/content/gdrive/MyDrive/Colab_Notebooks/drive/MyDrive/projected_sinkhorn/models')
# from models.resnet import *



# alpha = 0.02
# norm = 'linfinity'

# #model_name = '/content/gdrive/MyDrive/Colab_Notebooks/drive/MyDrive/projected_sinkhorn/checkpoints/cifar_dual_epoch_0_advacc_61.24_acc_99.91_OT_0.004999999888241291.pth'

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# best_acc = 0  # best test accuracy

# print('==> Preparing data..')
# transform_train = transforms.Compose([
#     transforms.RandomCrop(32, padding=4),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
#     # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2471, 0.2435, 0.2616)),
# ])

# transform_test = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2471, 0.2435, 0.2616)),
# ])
# mu = torch.Tensor((0.4914, 0.4822, 0.4465)).unsqueeze(-1).unsqueeze(-1).to(device)
# std = torch.Tensor((0.2471, 0.2435, 0.2616)).unsqueeze(-1).unsqueeze(-1).to(device)
# unnormalize = lambda x: x*std + mu
# normalize = lambda x: (x-mu)/std

# def cycle(iterable):
#     while True:
#         for x in iterable:
#             yield x

# # Return only images of certain class (eg. aeroplanes = class 0)
# def get_same_index(target, label):
#     label_indices = []
#     for i in range(len(target)):
#         if target[i] == label:
#             label_indices.append(i)
#     return label_indices



# trainset = torchvision.datasets.CIFAR10(root='./drive/MyDrive/projected_sinkhorn/dataset/', train=True, download=True, transform=transform_train)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True ) #CIFAR

# testset = torchvision.datasets.CIFAR10(root='./drive/MyDrive/projected_sinkhorn/dataset/', train=False, download=True, transform=transform_test)
# classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# # label_class = 9 # birds

# # train_indices = get_same_index( testset.targets, label_class)
# # catset = torch.utils.data.Subset(testset, train_indices)






# # Model architecture is from pytorch-cifar submodule
# print('==> Building model..')
# net = resnet18(pretrained=False)
# # net = net.to(device)
# # if device == 'cuda':
# #     net = torch.nn.DataParallel(net)
# #     cudnn.benchmark = True

# # # Load checkpoint.
# # print('==> Resuming from checkpoint..')
# # checkpoint = torch.load(model_name)

# #comment this for loading our adv trained model
# # for key in list(checkpoint['net'].keys()):
# #   new_key = key[7:]
# #   checkpoint['net'][new_key] = checkpoint['net'].pop(key)
# # net.load_state_dict(checkpoint['net'])

# # for key in list(checkpoint.keys()):
# #   new_key = key[7:]
# #   checkpoint[new_key] = checkpoint.pop(key)
# # net.load_state_dict(checkpoint,strict=False) # if name has 'robust'
# net = net.to(device)


In [154]:

# freeze parameters
# for pr in net.parameters(): 
#     pr.requires_grad = True

# for pr in net.layer4.parameters():
#   pr.requires_grad = True

# for pr in net.linear.parameters():
#   pr.requires_grad = True

# for pr in net.layer4.parameters():
#   print(pr.requires_grad)

# criterion = nn.CrossEntropyLoss()


# testloader = torch.utils.data.DataLoader(testset, batch_size= 200 , shuffle= True )
# def testcifar():
#     net.eval()
#     test_loss = 0
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for batch_idx, (inputs, targets) in enumerate(testloader):
#             inputs, targets = inputs.to(device), targets.to(device)
#             outputs = net(inputs)
#             loss = criterion(outputs, targets)

#             test_loss += loss.item()
#             _, predicted = outputs.max(1)
#             total += targets.size(0)
#             correct += predicted.eq(targets).sum().item()
#             # if batch_idx > 10:
#             #   break

#     acc = 100.*correct/total
#     print('cifar test accuracy is',acc)



# print('==> Evaluating model..')
# # testcifar()
# print('==> Attacking model..')


In [155]:
# p = 1
# kernel_size = 32
# C = wasserstein_cost(p=p, kernel_size= kernel_size, n = kernel_size )

# def test_attackcifar(): 
#     net.eval()
#     test_loss = 0
#     correct = 0
#     total = 0
#     batchOT = 0

#     for batch_idx, (inputs, targets) in enumerate(testloader):
        
#         inputs, targets = inputs.to(device), targets.to(device)
#         targets_attack = targets # should add or subtract for targeted attack
#         N = inputs.size(0)
#         start = timeit.timeit()
#         total += targets.size(0)
#         inputs_pgd,y, OT, _ = attack(torch.clamp(unnormalize(inputs),min=0), 
#                                  targets_attack, net,  C,total,batch_idx,targets,
#                                         p= p, alpha=alpha , norm=norm ,
#                                   maxiters= 10 ,thold = 10 , normalize=normalize ,
#                                  gamma = .5 ,
#                                  kernel_size= inputs.size(2) , delta = 0.003 , 
#                                  theeta = 100 , tau = .01 , 
#                                  eta = .001)
                        
#         outputs_pgd = net(normalize(inputs_pgd))
#         loss = criterion(outputs_pgd, y)
#         batchOT = batchOT + sum(OT)
#         test_loss += loss.item()
#         _, predicted = outputs_pgd.max(1)
        
        
        
        
#         AvgOT = batchOT/total
#         correct += predicted.eq(y).sum().item()
#         #print('total is.........',total)
#         print('Batch ' , batch_idx , 'Loss = ' , 
#               test_loss/(batch_idx+1) , 'Accuracy = ' , 
#               100.*correct/total, 'avg ot = ', AvgOT )
#         end = timeit.timeit()
#         # print('taargets:predicted',targets,predicted)
#         if batch_idx == 2:
#           break
#         acc = 100.*correct/total
#         AvgOT = batchOT/total
        
#     print('final acc = ' , 100.*correct/total, 'avg ot = ', AvgOT)
    
# # test_attackcifar()
# # !zip -r /content/gdrive/MyDrive/Colab_Notebooks/Images/CIFAR.zip /content/gdrive/MyDrive/Colab_Notebooks/Images/CIFAR

In [156]:
# !zip -r /content/gdrive/MyDrive/Colab_Notebooks/Images/CIFAR.zip /content/gdrive/MyDrive/Colab_Notebooks/Images/CIFAR

In [157]:
# # Model
# print('==> Building model..')
# net = ResNet18()
# net = net.to(device)
# if device == 'cuda':
#     net = torch.nn.DataParallel(net)
#     cudnn.benchmark = True


# def traincifar(epoch):
#     print('\nEpoch: %d' % epoch)
#     net.train()
#     train_loss = 0
#     correct = 0
#     nominal_correct = 0
#     total_epsilon = 0
#     total = 0
#     batchOT = 0
    
#     for batch_idx, (inputs, targets) in enumerate(trainloader):
#         inputs, targets = inputs.to(device), targets.to(device)

#         target_attacks = targets
#         # inputs_pgd,y, OT, _ = attack(torch.clamp(unnormalize(inputs),min=0), 
#         #                          target_attacks, net,  C, total,batch_idx,targets,
#         #                          p= p, alpha=alpha , norm=norm ,
#         #                          maxiters= 1 ,thold = 10 , normalize=normalize ,
#         #                          gamma = .5 ,
#         #                          kernel_size= 1024 , delta = 0.003 , 
#         #                          theeta = 100 , tau = .01 , 
#         #                          eta = 0.001 )
        
#         # print('Showing adv image at iter ',batch_idx)
#         # pixelsadv = inputs_pgd[0].cpu().numpy()
#         # print('max of adv',np.max(pixelsadv))
#         # print_cifar(pixelsadv)
#         # print('Showing original for inputs',batch_idx)
#         # pixelsadv = inputs[0].cpu().numpy()
#         # print('max of orig',np.max(pixelsadv))
#         # print_cifar(pixelsadv)

#         optimizer.zero_grad()
#         # outputs = net(normalize(inputs_pgd.detach()))
#         outputs = net(normalize(inputs))
#         loss = criterion(outputs, targets)
#         loss.backward()
#         optimizer.step()

#         with torch.no_grad(): 
#             outputs_nominal = net((inputs))
#             _, predicted_nominal = outputs_nominal.max(1)
#             nominal_correct += predicted_nominal.eq(targets).sum().item()

#             train_loss += loss.item()
#             _, predicted = outputs.max(1)
#             total += targets.size(0)
#             correct += predicted.eq(targets).sum().item()
#             # batchOT = batchOT + torch.sum(OT)
#             # AvgOT = batchOT/total

#         print(batch_idx, len(trainloader), 'Loss: %.3f | Adv Acc: %.3f%% (%d/%d) | Acc: %.3f%% (%d/%d)'
#             % (train_loss/(batch_idx+1), 100.*correct/total, correct, total, 
#                 100.*nominal_correct/total, nominal_correct, total))
#         # if batch_idx == 50:
#         #   break
#     # AvgOT = AvgOT.cpu().numpy()
#     # AvgOT = np.round(AvgOT,3)
#     advacc = np.round(100.*correct/total,2)
#     print(advacc)
#     acc = np.round(100.*nominal_correct/total,2)
#     state = {
#         'net': net.state_dict(),
#         'acc': acc,
#         'advacc': advacc,
#         # 'AvgOT': AvgOT,
#         'epoch': epoch,
#     }
#     torch.save(state, checkpoint_file.format(epoch,advacc,acc))
#     import shutil 
#     shutil.copy(checkpoint_file.format(epoch,advacc,acc), '/content/gdrive/MyDrive/projected_sinkhorn/checkpoints/')

# def test_attack(): 
#     net.eval()
#     test_loss = 0
#     correct = 0
#     total = 0
#     all_epsilons = []

#     for batch_idx, (inputs, targets) in enumerate(testloader):
#         inputs, targets = inputs.to(device), targets.to(device)
#         inputs_pgd,y, OT, _ = attack(torch.clamp(unnormalize(inputs),min=0), 
#                                  target_attacks, net,  C, total,batch_idx,targets,
#                                  p= p, alpha=alpha , norm=norm ,
#                                  maxiters= 2 ,thold = 30 , normalize=normalize ,
#                                  gamma = 2000 ,
#                                  kernel_size= 1024 , delta = 0.003 , 
#                                  theeta = 100 , tau = 5 , 
#                                  eta = 0.5 )

#         outputs_pgd = net(normalize(inputs_pgd))
#         loss = criterion(outputs_pgd, targets)

#         test_loss += loss.item()
#         _, predicted = outputs_pgd.max(1)
#         total += targets.size(0)
#         correct += predicted.eq(targets).sum().item()

#         # epsilons[predicted == targets] = -1
#         # all_epsilons.append(epsilons)

#         # print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d) | Avg epsilon: %.3f'
#         #     % (test_loss/(batch_idx+1), 100.*correct/total, correct, total, torch.cat(all_epsilons).float().mean().item()))
#         # print('\n')
#         print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d) '
#             % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
#         print('\n')
        

#     acc = 100.*correct/total
#     #all_epsilons = torch.cat(all_epsilons) 


# #test_attack()
# for epoch in range(start_epoch, start_epoch+10):
#     traincifar(epoch)
#     scheduler.step()
#     test_attackcifar()

# # testloader = torch.utils.data.DataLoader(testset, batch_size= 50 , shuffle= True )
# # testcifar()
# # #acc = 100.*correct/total
# # print('Saving..')
# # state = {
# #     'net': net.state_dict(),
# # }
# # # if not os.path.isdir(checkpoint_dir):
# # #     os.mkdir(checkpoint_dir)
# # torch.save(state, '/content/gdrive/MyDrive/Colab_Notebooks/drive/MyDrive/projected_sinkhorn/cifar10_dual.pth')


In [158]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
class CNN(nn.Module):
    """CNN."""

    def __init__(self):
        """CNN Builder."""
        super(CNN, self).__init__()

        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )


        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )


    def forward(self, x):
        """Perform forward."""
        
        # conv layers
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        x = self.fc_layer(x)

        return x


transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    #  std=[0.229, 0.224, 0.225]),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Normalize the test set same as training set without augmentation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    #  std=[0.229, 0.224, 0.225]),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./drive/MyDrive/projected_sinkhorn/dataset/', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=1, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./drive/MyDrive/projected_sinkhorn/dataset/', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=200, shuffle=False, num_workers=2)


criterion = nn.CrossEntropyLoss()
# net = resnet56_custom()
# checkpoint = torch.load(model_name)
# for key in list(checkpoint['state_dict'].keys()):
#   new_key = key[7:]
#   checkpoint['state_dict'][new_key] = checkpoint['state_dict'].pop(key)
# net.load_state_dict(checkpoint['state_dict'],strict=False)
# mu = torch.Tensor((0.485, 0.456, 0.406)).unsqueeze(-1).unsqueeze(-1).to(device)
# std = torch.Tensor((0.229, 0.224, 0.225)).unsqueeze(-1).unsqueeze(-1).to(device)

net = ResNet18()
checkpoint = torch.load(model_name)
for key in list(checkpoint['net'].keys()):
  new_key = key[7:]
  checkpoint['net'][new_key] = checkpoint['net'].pop(key)
net.load_state_dict(checkpoint['net'],strict=False)
mu = torch.Tensor((0.4914, 0.4822, 0.4465)).unsqueeze(-1).unsqueeze(-1).to(device)
std = torch.Tensor((0.2023, 0.1994, 0.2010)).unsqueeze(-1).unsqueeze(-1).to(device)

net=net.to(device)
optimizer = optim.Adam(net.parameters(), lr=.00001, weight_decay=.0005)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)

net = net.to(device)


for pr in net.parameters(): 
    pr.requires_grad = False


checkpoint_dir = '/content/gdrive/MyDrive/Colab_Notebooks/drive/MyDrive/projected_sinkhorn/checkpoints'
checkpoint_file = '{}/cifar_dual_epoch_{}_testacc_{}_attackacc_{}.pth'.format(checkpoint_dir,'{}','{}','{}')


unnormalize = lambda x: x*std + mu
normalize = lambda x: (x-mu)/std

criterion = nn.CrossEntropyLoss()

p = 1
kernel_size = 32
C = wasserstein_cost(p=p, kernel_size= kernel_size, n = kernel_size )
testacc=0 
attackacc=0
def test_attackcifar(): 
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    batchOT = 0

    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs.to(device), targets.to(device)
        targets_attack = targets # should add or subtract for targeted attack
        N = inputs.size(0)
        total += targets.size(0)
        inputs_pgd,y,  _ = attack(torch.clamp(unnormalize(inputs),min=0), 
                                 targets_attack, net,  C,total,batch_idx,targets,
                                        p= p, alpha=alpha , norm=norm ,
                                  maxiters= 50 ,thold = 10 , normalize=normalize ,
                                 gamma = .5 ,
                                 kernel_size= inputs.size(2) , delta = 0.003 , 
                                 theeta = 100 , tau = .01 , 
                                 eta = .001)
                        
        outputs_pgd = net(normalize(inputs_pgd))
        loss = criterion(outputs_pgd, y)
        test_loss += loss.item()
        _, predicted = outputs_pgd.max(1)
        
        correct += predicted.eq(y).sum().item()
        #print('total is.........',total)
        print('Batch ' , batch_idx , 'Loss = ' , 
              test_loss/(batch_idx+1) , 'Accuracy = ' , 
              100.*correct/total)
        # print('taargets:predicted',targets,predicted)
        # if batch_idx == 2:
        #   break
        attackacc = 100.*correct/total
        
    print('final acc = ' , 100.*correct/total)
    return attackacc

def testcifar():
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            # if batch_idx > 10:
            #   break

    testacc = 100.*correct/total
    print('cifar test accuracy is',testacc)
    return testacc


print('==> Evaluating model..')
testcifar()
print('==> Attacking model..')

checkpoint = torch.load(model_name)
for key in list(checkpoint['net'].keys()):
  new_key = key[7:]
  checkpoint['net'][new_key] = checkpoint['net'].pop(key)
net.load_state_dict(checkpoint['net'],strict=False)

testcifar()
def traincifar(testacc,attackacc,epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    nominal_correct = 0
    total_epsilon = 0
    total = 0
    batchOT = 0
    
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)

        target_attacks = targets
        inputs_pgd,y, _ = attack(torch.clamp(unnormalize(inputs),min=0), 
                                 target_attacks, net,  C, total,batch_idx,targets,
                                 p= p, alpha=alpha , norm=norm ,
                                 maxiters= 5 ,thold = 10 , normalize=normalize ,
                                 gamma = .5 ,
                                 kernel_size= 1024 , delta = 0.003 , 
                                 theeta = 100 , tau = .01 , 
                                 eta = 0.001 )
        
        # print('Showing adv image at iter ',batch_idx)
        # pixelsadv = inputs_pgd[0].cpu().numpy()
        # print('max of adv',np.max(pixelsadv))
        # print_cifar(pixelsadv)
        # print('Showing original for inputs',batch_idx)
        # pixelsadv = inputs[0].cpu().numpy()
        # print('max of orig',np.max(pixelsadv))
        # print_cifar(pixelsadv)

        optimizer.zero_grad()
        # outputs = net(normalize(inputs_pgd.detach()))
        outputs = net(normalize(inputs_pgd))
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        with torch.no_grad(): 
            outputs_nominal = net(normalize(inputs))
            _, predicted_nominal = outputs_nominal.max(1)
            nominal_correct += predicted_nominal.eq(targets).sum().item()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            # batchOT = batchOT + torch.sum(OT)
            # AvgOT = batchOT/total

        print(batch_idx, len(trainloader), 'Loss: %.3f | Adv Acc: %.3f%% (%d/%d) | Acc: %.3f%% (%d/%d)'
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total, 
                100.*nominal_correct/total, nominal_correct, total))
        # if batch_idx == 50:
        #   break
    # AvgOT = AvgOT.cpu().numpy()
    # AvgOT = np.round(AvgOT,3)
   
    acc = np.round(100.*nominal_correct/total,2)
    state = {
        'net': net.state_dict(),
        'testacc': testacc,
        'attackacc': attackacc,
        # 'AvgOT': AvgOT,
        'epoch': epoch,
    }
    torch.save(state, checkpoint_file.format(epoch,testacc,attackacc))
    import shutil 
    shutil.copy(checkpoint_file.format(epoch,testacc,attackacc), '/content/gdrive/MyDrive/projected_sinkhorn/checkpoints/')

# for epoch in range(start_epoch, start_epoch+1):
#     if epoch%3 == 0:
#       # testacc = testcifar()
#       attackacc = test_attackcifar()
    # traincifar(testacc,attackacc,epoch)
#     # scheduler.step()
    
!zip -r /content/gdrive/MyDrive/Colab_Notebooks/Images/CIFAR.zip /content/gdrive/MyDrive/Colab_Notebooks/Images/CIFAR    


Extracting ./drive/MyDrive/projected_sinkhorn/dataset/cifar-10-python.tar.gz to ./drive/MyDrive/projected_sinkhorn/dataset/
Files already downloaded and verified


KeyError: ignored